In [2]:
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI\\research'

In [3]:
import os
os.chdir("../")
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI'

In [4]:
from sklearn.model_selection import train_test_split
import keras

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    METRIC_FILE: Path
    model: Path

In [6]:
from Sentiment.constants import *
from Sentiment.utils.common import read_yaml, create_directories

In [7]:
from Sentiment.constants import *
from Sentiment.utils.common import read_yaml, create_directories
from Sentiment.entity.config_entity import DataIngestionConfig, DataValidationConfig, DataTransformationConfig, ModelTrainerConfig, ModelEvaluationConfig
from Sentiment import logger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES,
        )

        return data_validation_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_present_dir,
            data_path=config.data_present_path,
            save_dir=config.root_dir,
            save_path=config.data_path
        )
        return data_transformation_config
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            vocab_size = params.vocab_size,
            oov_tok = params.oov_tok,
            embedding_dim = params.embedding_dim,
            max_length = params.max_length, # choose based on statistics, for example 150 to 200
            padding_type =  params.padding_type,
            trunc_type = params.trunc_type,
            units = params.units, #units: The number of hidden units in the layer.
            hidden_dense = params.hidden_dense,
            last_dense = params.last_dense,
            loss = params.loss,
            optimizer = params.optimizer,
            metrics = params.metrics,
            num_epochs = params.num_epochs,
            verbose = params.verbose,
            validation_split = params.validation_split,
            dense_layers = params.dense_layers,
            last_layer = params.last_layer
        )

        return model_trainer_config
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            METRIC_FILE = config.METRIC_FILE,
            model = config.model
        )

        return model_evaluation_config

In [8]:
import os
from Sentiment import logger
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def loadModel(self):
        model = keras.models.load_model(self.config.model)
        return model
    
    def predictTest(self,testSentences,model):
        
        return model.predict(testSentences)
    
    def sentimentClassify(self,predictions):
        prediction = []
        for i in predictions:
            if i > 50:
                prediction.append(1)
            else:
                prediction.append(0)
        return prediction
    
    def TextToNumeric(self,testSentences):
        '''
        Converting the text to numeric
        '''
        try:
            vocab_size = 3000 # choose based on statistics
            oov_tok = ''
            max_length = 200
            padding_type = 'post'
            tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
            tokenizer.fit_on_texts(testSentences)
            word_index = tokenizer.word_index
            # convert Test dataset to sequence and pad sequences
            testSequences = tokenizer.texts_to_sequences(testSentences)
            testPadded = pad_sequences(testSequences, padding=padding_type, maxlen=max_length)
            return testPadded
        except Exception as e:
            raise e
    
    def EvaluationMetrics(self,predictions,testLabels):
        try:
            accuracyScore = accuracy_score(testLabels,predictions)
            recallScore = recall_score(testLabels,predictions)
            precisionScore = precision_score(testLabels,predictions)
            f1Score = f1_score(testLabels,predictions)
            return accuracyScore,recallScore,precisionScore,f1Score
        except Exception as e:
            raise e
        
    def ModelEvaluationStatus(self,accuracyScore,recallScore,precisionScore,f1Score):
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","model_evaluation"))

            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    Evaluation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Evaluation status: {Evaluation_status}")
                else:
                    Evaluation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Evaluation status: {Evaluation_status}"+",")
                        f.write(f"accuracyScore: {accuracyScore}"+",")
                        f.write(f"recallScore: {recallScore}"+",")
                        f.write(f"precisionScore: {precisionScore}"+",")
                        f.write(f"f1Score: {f1Score}"+",")

            return validation_status
        
        except Exception as e:
            raise e

In [11]:
from Sentiment.components.model_trainer import ModelTrainer
from Sentiment.components.model_evaluation import ModelEvaluation

In [12]:
from Sentiment.config.configuration import ConfigurationManager

In [25]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    logger.info(f"data loading Started")
    sentiData = model_trainer_config.loadData()
    logger.info(f"data loading Completed")
    logger.info(f"data Lowercase convertion Started")
    sentiData = model_trainer_config.convertToLower(sentiData)
    logger.info(f"data Lowercase convertion Completed")
    logger.info(f"data Splitting Started")
    trainSentences, testSentences, trainLabels, testLabels = model_trainer_config.splitData(sentiData)
    logger.info(f"data Splitting Completed")
    logger.info(f"tokenizing the Text Started")
    testPadded = model_evaluation_config.TextToNumeric(testSentences)
    logger.info(f"tokenizing the Text Completed")
    logger.info(f"Model Loading")
    model = model_evaluation_config.loadModel()
    logger.info(f"Model Loaded")
    logger.info(f"Predicting Test Data")
    predictions = model_evaluation_config.predictTest(testPadded,model)
    logger.info(f"Predicting Test Data Completed")
    logger.info(f"Predicting classification Started")
    prediction = model_evaluation_config.sentimentClassify(predictions)
    logger.info(f"Predicting classification Completed")
    logger.info(f"EvaluationMetrics Started")
    accuracyScore,recallScore,precisionScore,f1Score = model_evaluation_config.EvaluationMetrics(prediction,testLabels)
    logger.info(f"EvaluationMetrics Completed")
    status = model_evaluation_config.ModelEvaluationStatus(accuracyScore,recallScore,precisionScore,f1Score)
    print(status)
    logger.info(f"EvaluationMetrics Saved")
except Exception as e:
    raise e

[2024-01-06 02:14:00,600: INFO: common: yaml file: {yaml_file}]
[2024-01-06 02:14:00,603: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-06 02:14:00,605: INFO: common: yaml file: {yaml_file}]
[2024-01-06 02:14:00,606: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-06 02:14:00,607: INFO: common: created directory at: artifacts]
[2024-01-06 02:14:00,608: INFO: common: created directory at: artifacts/model_trainer]
[2024-01-06 02:14:00,609: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-06 02:14:00,610: INFO: 3575743728: data loading Started]
[2024-01-06 02:14:13,578: INFO: 3575743728: data loading Completed]
[2024-01-06 02:14:13,579: INFO: 3575743728: data Lowercase convertion Started]
[2024-01-06 02:14:13,832: INFO: model_trainer: Data converted to lowercase]
[2024-01-06 02:14:13,833: INFO: 3575743728: data Lowercase convertion Completed]
[2024-01-06 02:14:13,834: INFO: 3575743728: data Splitting Started]
[2024-01-

c:\Users\akish\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from sklearn import metrics
recallScore = metrics.f1_score(testLabels,prediction,average='weighted',zero_division=0)

In [23]:
recallScore

0.324263889291608